In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from boruta import BorutaPy as bp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
import time
from sklearn.preprocessing import MinMaxScaler

/apps/python-2.7.14/lib/python2.7/site-packages/scikit_learn-0.19.2-py2.7-linux-x86_64.egg/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data = pd.read_csv('specPhotoDR10v2_GauthamG_0.csv')

In [3]:
data.dropna(inplace = True)

In [4]:
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])
data.head()

,specObjID,objid,ra,dec,photoz,photoz_err,specz,specz_err,dered_u,dered_g,...,expPhi_g,expAB_r,expRad_r,expPhi_r,expAB_i,expRad_i,expPhi_i,expAB_z,expRad_z,expPhi_z
0,0.081865,0.002829,0.154768,0.128969,0.297002,0.999681,0.043863,0.001021,0.533379,0.480263,...,0.464509,0.824348,0.013380,0.479943,0.796099,0.013494,0.480844,0.606807,0.010235,0.506055
1,0.081868,0.002829,0.156205,0.129019,0.584260,0.999817,0.064740,0.001021,0.479019,0.490616,...,0.540118,0.594466,0.013163,0.515143,0.610589,0.013029,0.511761,0.279182,0.012368,0.545185
2,0.081493,0.002877,0.141090,0.136944,0.152211,0.999683,0.022515,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
3,0.081494,0.002877,0.141090,0.136944,0.152211,0.999683,0.022520,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
6,0.081493,0.002877,0.141406,0.138006,0.606950,0.999776,0.001497,0.001021,0.600054,0.507319,...,0.578584,0.172073,0.025074,0.583177,0.338253,0.005512,0.543976,0.197601,0.006683,0.507553


In [5]:
data.dropna(inplace = True)

In [6]:
data = data.drop(['specObjID', 'objid'], axis = 1)

In [7]:
XErr = []
sub1 = 'err'
for a in list(data.columns):
  if sub1 in a:
    XErr.append(a)
sub2 = 'Err'
for a in list(data.columns):
  if sub2 in a:
    XErr.append(a)

In [8]:
data = data.drop(XErr, axis = 1)

In [9]:
data_origi = data

In [10]:
data = data.drop(['specType', 'photpType', 'specz'], axis = 1)

In [11]:
RFR = RandomForestRegressor()
X = data.drop(['photoz'], axis = 1)
y = data['photoz']
names = list(X.columns)

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [24]:
def rfr_model(X, y):
#         Grid Search
        gsc = GridSearchCV(estimator = RandomForestRegressor(), param_grid = {'max_depth' : range(3, 7), 'n_estimators' : (10, 50, 100, 1000)}, cv = 5, scoring = 'neg_mean_squared_error', verbose = 0, n_jobs = -1)
        grid_result = gsc.fit(X, y)
        best_params = grid_result.best_params_
        
        rfr = RandomForestRegressor(max_depth=best_params['max_depth'], n_estimators=best_params['n_estimators'], random_state = False, verbose = False)
#          5-fold CV        
        scores = cross_val_score(rfr, X, y, cv = 5, scoring = 'neg_mean_squared_error')
        return scores

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [26]:
rfr_model(X_train, y_train)

array([-0.00293879, -0.00293265, -0.00292337, -0.00294917, -0.00295836])

 Looks like there is no need of cross validation as the default model performs with a very similar score and crossvalidating for each sample/Algorithm takes > 6hours for 3-4 sets of parameters on an average, gain in accuracy does not seem good enough for amount of runtime required to crossvalidate.